In [102]:
# # create empty DataFrame df and append row
# my_cols = ['date', 'days_lookback', 'syms_freq', 'symbols']
# # Creating Empty DataFrame and Storing it in variable df
# df_model_top_picks = pd.DataFrame(columns=my_cols)
# df_model_top_picks

In [103]:
import pandas as pd
from myUtils import pickle_load, pickle_dump
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 900)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_picks  = f'df_picks'
fp_df_model_top_picks = f'df_model_top_picks'

verbose = True
# verbose = False

In [104]:
df = pickle_load(path_data_dump, fp_df_picks)
df = df.drop_duplicates(subset=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], keep='last')
df = df.sort_values(by=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], ascending=False)
df = df.reset_index(drop=True)
pickle_dump(df, path_data_dump, fp_df_picks)
print(f'len(df): {len(df)}')
print(df.head())

len(df): 48
  date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14 sym_freq_13 sym_freq_12 sym_freq_11 sym_freq_10 sym_freq_9      sym_freq_8 sym_freq_7              sym_freq_6                     sym_freq_5                     sym_freq_4                     sym_freq_3 sym_freq_2
0        2023-04-18                 120      [30, 60, 120]          []          []          []          []          []          []         []         ['SHV']    ['ELF']         ['FTSM', 'NVO']  ['AMPH', 'COTY', 'GE', 'NV...   ['AU', 'LNTH', 'SGEN', 'TA']  ['ACLS', 'ANET', 'AZN', 'B...         []
1        2023-04-18                 120  [15, 30, 60, 120]          []          []          []          []          []          []         []         ['SHV']    ['ELF']  ['FTSM', 'NVO', 'WWE']  ['AMPH', 'COTY', 'GE', 'NV...  ['AU', 'LNTH', 'MARA', 'SG...  ['ACLS', 'ANET', 'AZN', 'B...         []
2        2023-04-17                 120      [30, 60, 120]          []          []          []   

In [105]:
from itertools import product
from ast import literal_eval

my_cols = ['sym_freq_12', 'sym_freq_9', 'sym_freq_8']
my_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

my_dates = []
for my_days_lookback in my_days_lookbacks:
  my_row = (df['days_lookbacks'] == str(my_days_lookback))
  l_dates = df.loc[my_row, 'date_end_df_train'].tolist()     
  my_dates.append(l_dates)

my_dates = [val for sublist in my_dates for val in sublist]
my_dates = sorted(list(set(my_dates)), reverse=True)

d_lbk_freq = list(product(my_dates, my_days_lookbacks, my_cols))

picks = []
prev_date = ''
# print(f'Model predicts the best performing symbols are:\n[30, 60, 120]        sym_freq_9\n[15, 30, 60, 120]    sym_freq_12\n[30, 60, 120]        sym_freq_8')
print(f'Model predicts the best performing symbols are:')
print(f'1st    {"[30, 60, 120]":<20}{"sym_freq_9":<14}{"days_eval = 4":<20}')
print(f'2nd    {"[15, 30, 60, 120]":<20}{"sym_freq_12":<14}{"days_eval = 5":<20}')
print(f'3rd    {"[30, 60, 120]":<20}{"sym_freq_8":<14}{"days_eval = 4":<20}')
for i, item in enumerate(d_lbk_freq):
  _date, my_days_lookback, my_col = item[0], item[1], item[2]
  my_row = (df['date_end_df_train'] == _date) & (df['days_lookbacks'] == str(my_days_lookback))  
  list_in_df = df.loc[my_row, my_col]
  my_str = list_in_df.tolist()[0]
  my_list = literal_eval(my_str)
  blank = ' '
  if _date != prev_date:
    if prev_date == '':
      print('\nModel Results:')  
      print(f'{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')
    else:
      print(f'\n{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}') 
  else:
    print(f'{i:<6}{blank:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')

  picks.append((i, my_list))
  prev_date = _date

Model predicts the best performing symbols are:
1st    [30, 60, 120]       sym_freq_9    days_eval = 4       
2nd    [15, 30, 60, 120]   sym_freq_12   days_eval = 5       
3rd    [30, 60, 120]       sym_freq_8    days_eval = 4       

Model Results:
0     2023-04-18    [30, 60, 120]        sym_freq_12: []
1                   [30, 60, 120]        sym_freq_9 : []
2                   [30, 60, 120]        sym_freq_8 : ['SHV']
3                   [15, 30, 60, 120]    sym_freq_12: []
4                   [15, 30, 60, 120]    sym_freq_9 : []
5                   [15, 30, 60, 120]    sym_freq_8 : ['SHV']

6     2023-04-17    [30, 60, 120]        sym_freq_12: []
7                   [30, 60, 120]        sym_freq_9 : []
8                   [30, 60, 120]        sym_freq_8 : ['ELF', 'SHV']
9                   [15, 30, 60, 120]    sym_freq_12: []
10                  [15, 30, 60, 120]    sym_freq_9 : []
11                  [15, 30, 60, 120]    sym_freq_8 : ['ELF', 'SHV']

12    2023-04-14    [30, 60, 1

In [106]:
model_top_picks = [([30, 60, 120], 'sym_freq_9'), ([15, 30, 60, 120], 'sym_freq_12'), ([30, 60, 120], 'sym_freq_8')]

In [107]:
df_model_top_picks = pickle_load(path_data_dump, fp_df_model_top_picks)

z_items   = zip(d_lbk_freq, picks)
for i, item in enumerate(z_items):
  _date, _days_lookback, _sym_freq, _syms = item[0][0], item[0][1], item[0][2], item[1][1]
  print(i, _date, _days_lookback, _sym_freq, _syms)
  dlb_sym_freq = (_days_lookback, _sym_freq)
  if dlb_sym_freq in model_top_picks and _syms:
    row_add = [_date, _days_lookback, _sym_freq, _syms]
    df_model_top_picks.loc[len(df_model_top_picks)] = row_add
    print(f'added row to df_model_top_picks:\n{row_add}\n')


0 2023-04-18 [30, 60, 120] sym_freq_12 []
1 2023-04-18 [30, 60, 120] sym_freq_9 []
2 2023-04-18 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-18', [30, 60, 120], 'sym_freq_8', ['SHV']]

3 2023-04-18 [15, 30, 60, 120] sym_freq_12 []
4 2023-04-18 [15, 30, 60, 120] sym_freq_9 []
5 2023-04-18 [15, 30, 60, 120] sym_freq_8 ['SHV']
6 2023-04-17 [30, 60, 120] sym_freq_12 []
7 2023-04-17 [30, 60, 120] sym_freq_9 []
8 2023-04-17 [30, 60, 120] sym_freq_8 ['ELF', 'SHV']
added row to df_model_top_picks:
['2023-04-17', [30, 60, 120], 'sym_freq_8', ['ELF', 'SHV']]

9 2023-04-17 [15, 30, 60, 120] sym_freq_12 []
10 2023-04-17 [15, 30, 60, 120] sym_freq_9 []
11 2023-04-17 [15, 30, 60, 120] sym_freq_8 ['ELF', 'SHV']
12 2023-04-14 [30, 60, 120] sym_freq_12 []
13 2023-04-14 [30, 60, 120] sym_freq_9 []
14 2023-04-14 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-14', [30, 60, 120], 'sym_freq_8', ['SHV']]

15 2023-04-14 [15, 30, 60, 120] sym_freq_12 

In [108]:
df_model_top_picks = df_model_top_picks.astype(str)
df_model_top_picks = df_model_top_picks.drop_duplicates(keep='last')
df_model_top_picks = df_model_top_picks.sort_values(by=['date', 'days_lookback', 'syms_freq'], ascending=False)
df_model_top_picks = df_model_top_picks.reset_index(drop=True)
pickle_dump(df_model_top_picks, path_data_dump, fp_df_model_top_picks)
print(f'len(df_model_top_picks): {len(df_model_top_picks)}')
print(f'{fp_df_model_top_picks}:\n{df_model_top_picks}')

len(df_model_top_picks): 29
df_model_top_picks:
          date  days_lookback   syms_freq          symbols
0   2023-04-18  [30, 60, 120]  sym_freq_8          ['SHV']
1   2023-04-17  [30, 60, 120]  sym_freq_8   ['ELF', 'SHV']
2   2023-04-14  [30, 60, 120]  sym_freq_8          ['SHV']
3   2023-04-13  [30, 60, 120]  sym_freq_8          ['SHV']
4   2023-04-12  [30, 60, 120]  sym_freq_8          ['SHV']
5   2023-04-11  [30, 60, 120]  sym_freq_8          ['SHV']
6   2023-04-10  [30, 60, 120]  sym_freq_8          ['SHV']
7   2023-04-06  [30, 60, 120]  sym_freq_8          ['SHV']
8   2023-04-05  [30, 60, 120]  sym_freq_9          ['SHV']
9   2023-04-04  [30, 60, 120]  sym_freq_8  ['NVDA', 'SHV']
10  2023-04-03  [30, 60, 120]  sym_freq_8   ['GE', 'NVDA']
11  2023-03-31  [30, 60, 120]  sym_freq_8         ['NVDA']
12  2023-03-30  [30, 60, 120]  sym_freq_9          ['SHV']
13  2023-03-30  [30, 60, 120]  sym_freq_8         ['NVDA']
14  2023-03-29  [30, 60, 120]  sym_freq_9          ['SHV']
15  2023

In [109]:
df_mp = df_model_top_picks.copy()
df_close = pickle_load(path_data_dump, 'df_close_clean')

In [110]:
def get_NYSE_date_n(date, dates_NYSE, n, verbose=False):

  if date in dates_NYSE:
    idx_last = len(dates_NYSE) - 1  # last index of dates_NYSE
    idx_date = dates_NYSE.get_loc(date)  # index of date
    idx_date_n = idx_date + n  # index of date_n
    if verbose:
      print(f"date: {date} is in dates_NYSE,  date's position in dates_NYSE is: {idx_date} of {idx_last},  n: {n},  idx_date_n: {idx_date_n},")
    if 0 <= idx_date_n <= idx_last:  # idx_date_n is within bounds of dates_NYSE
      date_n = dates_NYSE[idx_date_n]
      if verbose:      
        print(f'idx_date_n: {idx_date_n} is wiithin bounds of dates_NYSE ("0 to {idx_last}), date_n: {date_n}"\n')
    else:  # idx_date_n is out-of-bounds of dates_NYSE
      date_n = None   
      if verbose:  
        print(f'idx_date_n: {idx_date_n} is out-of-bounds of dates_NYSE ("0 to {idx_last})"\n')
  else:  # date not in dates_NYSE
      date_n = None  
      if verbose:  
        print(f'date: {date} is not in dates_NYSE\n')

  return date_n  

In [111]:
def get_portf_buy(df_close, date, str_symbols, portf_target, verbose=False):
    import numpy as np
    from ast import literal_eval    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
    # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_close.loc[date][l_syms].values  
    sym_cnt = len(l_syms)  # number of symbols
    amt_per_sym = portf_target / sym_cnt  # target dollar investment in each symbol
    ar_sym_share = np.floor(amt_per_sym / ar_price)  # array of shares for each symbol
    ar_sym_amt = ar_price * ar_sym_share  # array of actual dollar amount invested in each symbol
    portf_value = sum(ar_sym_amt)  # total actual portfolio value
    if verbose:
        # print(date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value)    
        print(f'{date = }, {l_syms = }, {ar_price = }, {ar_sym_share = }, {ar_sym_amt = }, {portf_value = }')            
        print(f'{date} {portf_value = }')
    return date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value

In [112]:
def get_SPY_buy(df_close, date, portf_target, symbol="SPY", verbose=False):   
    import numpy as np
    price = df_close.loc[date][symbol]    
    share = np.floor(portf_target / price)  # array of shares for each symbol
    value = price * share  # array of actual dollar amount invested in each symbol
    if verbose:
        # print(date, symbol, price, share, value)
        print(f'{date = }, {symbol = }, {price = }, {share = }, {value = }')                        
        print(f'{date} {symbol} {value = }')
    return date, price, share, value

In [113]:
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
dates_NYSE_2013_2023 = nyse.valid_days(start_date='2013-01-01', end_date='2023-12-31').strftime('%Y-%m-%d')
print(f'len(dates_NYSE_2013_2023): {len(dates_NYSE_2013_2023)}')
print(f'dates_NYSE_2013_2023[0]:  {dates_NYSE_2013_2023[0]}, dates_NYSE_2013_2023[-1]: {dates_NYSE_2013_2023[-1]}')
print(f'{dates_NYSE_2013_2023}') 

len(dates_NYSE_2013_2023): 2768
dates_NYSE_2013_2023[0]:  2013-01-02, dates_NYSE_2013_2023[-1]: 2023-12-29
Index(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07', '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11', '2013-01-14', '2013-01-15',
       ...
       '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27', '2023-12-28', '2023-12-29'], dtype='object', length=2768)


In [114]:
import numpy as np
# from ast import literal_eval
z_date_syms = zip(df_mp.date, df_mp.symbols)

idx_close = df_close.index.strftime('%Y-%m-%d')

n = 1  # n=1 is the buy date
# n = 4  # n=4 is the sell date

date_buy = []  # buy date of portfolio
shares_syms = []  # lists of shares of each symbol brought on date_buy
value_portf = [] # list of porfolio value on date_buy
shares_SPY = []  # list of shares of SPY brought on date_buy
value_SPY = []  # list of value of SPY shares on date_buy 
for date, syms in z_date_syms:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=verbose)

    if next_date_n in idx_close:
        close_date_n = next_date_n
        print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is in df_close')
        print(f'picks for {date}: {syms}')      
    else:
        close_date_n = None
        print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is not in df_close')
        print(f'picks for {date}: {syms}')        

    if close_date_n is None:
        p_date = None
        p_l_syms = None
        p_ar_price = None
        p_ar_sym_share = None
        p_ar_sym_amt = None
        p_portf_value = None  # data not available in df_close
        
        s_date = None
        s_price = None
        s_share = None
        s_value = None  # data not available in df_close

        date_buy.append(p_date)
        shares_syms.append(p_l_syms)
        value_portf.append(p_portf_value)
        shares_SPY.append(s_share)
        value_SPY.append(s_value)    
        
        print(f"No data for next_date_n {next_date_n}, pick's portf value = None")
        print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

    else:    
        p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
            get_portf_buy(df_close, next_date_n, syms, portf_target=1000, verbose=verbose)
        s_date, s_price, s_share, s_value = \
            get_SPY_buy(df_close, next_date_n, portf_target=1000, verbose=verbose)
                
        date_buy.append(p_date)
        shares_syms.append(p_ar_sym_share)
        value_portf.append(p_portf_value)
        shares_SPY.append(s_share)
        value_SPY.append(s_value)

        print(f"next_date_n pick's portf value = {p_portf_value}")
        print(f'next_date_n SPY portf value =    {s_value}')

    print('='*20, '\n')       
    

date: 2023-04-18 is in dates_NYSE,  date's position in dates_NYSE is: 2590 of 2767,  n: 1,  idx_date_n: 2591,
idx_date_n: 2591 is wiithin bounds of dates_NYSE ("0 to 2767), date_n: 2023-04-19"

date: 2023-04-18,  n: 1,  next_date_n: 2023-04-19 is not in df_close
picks for 2023-04-18: ['SHV']
No data for next_date_n 2023-04-19, pick's portf value = None
No data for next_date_n 2023-04-19, SPY portf value =    None

date: 2023-04-17 is in dates_NYSE,  date's position in dates_NYSE is: 2589 of 2767,  n: 1,  idx_date_n: 2590,
idx_date_n: 2590 is wiithin bounds of dates_NYSE ("0 to 2767), date_n: 2023-04-18"

date: 2023-04-17,  n: 1,  next_date_n: 2023-04-18 is in df_close
picks for 2023-04-17: ['ELF', 'SHV']
date = '2023-04-18', l_syms = ['ELF', 'SHV'], ar_price = array([ 92.58000183, 110.29000092]), ar_sym_share = array([5., 4.]), ar_sym_amt = array([462.90000916, 441.16000366]), portf_value = 904.0600128173828
2023-04-18 portf_value = 904.0600128173828
date = '2023-04-18', symbol = 'SPY'

In [115]:
# def get_next_date(df_close, date):
#     """Get the next date from df_close. date has to in df_close.index.

#     Edge case: The function won't return the first date in df_close as the
#     next trading date, if date is a trading date just before the first date in df_close.
#     """

#     len_df_close = len(df_close)
#     idx_dates = df_close.index.strftime('%Y-%m-%d')  # date index of df_close    

#     if date in idx_dates:
#         idx_next_date = idx_dates.get_loc(date) + 1  # get index position of next date

#         if idx_next_date <= (len_df_close - 1):  # next date's index is within bound of df_close
#             next_date = idx_dates[idx_dates.get_loc(date) + 1]  # get the next date
#             return next_date  
#         else:  # idx_next_date is out-of_bounds df_close
#             print(f"{date} is the last date in df_close. No data for next day")
#             return None    
#     else:     
#         # no data from df_close
#         print(f'no data for {date} in df_close')
#         return None

In [116]:
# def get_symbols_close(my_date, df_close):
#   # get closing price for symbols for my_date, i.e. a row of df_close for my_date
#   # return symbols' closing price as a series
#   if my_date in df_close.index:  
#     iloc_my_date = df_close.index.get_loc(my_date)
#     print(f'len(df_close): {len(df_close)}, my_date: {my_date}, iloc_my_date: {iloc_my_date}')      
#     symbols_close = df_close.iloc[iloc_my_date]    
#   else:
#     msg_stop = f'ERROR: my_date: {my_date} is not a date in df_close.index\ndf_close.index:\n{df_close.index}'
#     raise SystemExit(msg_stop)
#   return symbols_close

In [117]:
def calc_portf_shares(df_close, date, str_symbols, portf_target):
  # calculate number of shares to buy for symbols in str_symbols to meet port_target
  import numpy as np
  from ast import literal_eval    
  l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
  # array of closing prices corresponding to symbols in l_syms    
  ar_price = df_close.loc[date][l_syms].values  
  sym_cnt = len(l_syms)  # number of symbols
  amt_per_sym = portf_target / sym_cnt  # target dollar investment in each symbol
  ar_shares = np.floor(amt_per_sym / ar_price)  # array of shares for each symbol
  return ar_shares

In [118]:
def calc_portf_value(df_close, date, str_symbols, ar_shares, verbose=False):
    import numpy as np
    from ast import literal_eval    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
    # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_close.loc[date][l_syms].values  
    ar_value = ar_price * ar_shares  # array of actual dollar amount invested in each symbol
    portf_value = sum(ar_value)  # total actual portfolio value
    if verbose:
        print(f'{date = }, {l_syms = }, {ar_price = }, {ar_shares = }, {ar_value = }, {portf_value = }')            
        print(f'{date} {portf_value = }')
    return date, l_syms, ar_price, ar_shares, ar_value, portf_value

In [119]:
def get_next_date_n(df_close, date, n=1):

    """Get the next n(default=1) date from df_close. date has to in df_close.index.

    Edge case: The function won't return the first date in df_close as the
    next trading date, if date is a trading date just before the first date in df_close.
    """

    len_df_close = len(df_close)
    idx_dates = df_close.index.strftime('%Y-%m-%d')  # date index of df_close    

    if date in idx_dates:
        idx_next_date = idx_dates.get_loc(date) + n  # get index position of next date
        print(f'n: {n},  idx_next_date: {idx_next_date}')

        if idx_next_date >= 0:  # if idx_nex_date is negative, stop index loop back from index[0] to index[-1]    
            if idx_next_date >= (len_df_close - n):  # next date's index is within bound of df_close
                next_date = idx_dates[idx_dates.get_loc(date) + n]  # get the next date
                return next_date
        
              
            else:  # idx_next_date is out-of_bounds df_close
                print(f"{date} is the last date in df_close. No data for next {n} day")
                return None    
        
        
        else:  # idx_next_date is negative, no data in df_close
            # no data from df_close
            print(f'idx_next_date: {idx_next_date} is negative, no data for {n} days from {date} in df_close')

    else:     
        # no data from df_close
        print(f'no data for {date} in df_close')
        return None

In [120]:
def is_date_in_close(date, df_close):
  idx_close = df_close.index.strftime('%Y-%m-%d')
  if date in idx_close:
    return date
  else:
    return None

In [121]:
def calc_portf_value_date_buy(my_dates, my_symbols, df_close, portf_target, n, verbose=False):
  
  z_date_syms = zip(my_dates, my_symbols)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, syms in z_date_syms:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_close)

    if close_date_n is None:
      p_date = None
      p_ar_shares = None
      p_portf_value = None  # set to None when data are not available in df_close
      SPY_shares = None
      SPY_value = None  # set to None when data are not available in df_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        print(f'No data for close_date_n {close_date_n}, SPY portf value =    None')

    else:    
      p_ar_shares = calc_portf_shares(df_close, close_date_n, syms, portf_target)
      p_date, l_syms, ar_price, ar_shares, ar_value, p_portf_value = \
        calc_portf_value(df_close, close_date_n, syms, p_ar_shares, verbose)

      syms = str(['SPY'])
      SPY_shares = calc_portf_shares(df_close, close_date_n, syms, portf_target)
      date, l_syms, ar_price, ar_shares, ar_value, SPY_value = \
        calc_portf_value(df_close, close_date_n, syms, SPY_shares, verbose)

      if verbose:
        print(f"close_date_n pick's portf value = {p_portf_value}")
        print(f'close_date_n SPY portf value =    {SPY_value}')

    date_exec.append(p_date)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_portf_value)
    shares_SPY.append(SPY_shares)
    value_SPY.append(SPY_value)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [122]:
# def get_portf_value(my_dates, my_symbols, df_close, n):
  
#   z_date_syms = zip(my_dates, my_symbols)

#   date_buy = []  # buy date of portfolio
#   shares_syms = []  # lists of shares of each symbol brought on date_buy
#   value_portf = [] # list of porfolio value on date_buy
#   shares_SPY = []  # list of shares of SPY brought on date_buy
#   value_SPY = []  # list of value of SPY shares on date_buy 

#   for date, syms in z_date_syms:
#     next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=verbose)
#     close_date_n = is_date_in_close(next_date_n, df_close)

#     if close_date_n is None:
#       p_date = None
#       p_ar_sym_share = None
#       p_portf_value = None  # set to None when data are not available in df_close
#       s_share = None
#       s_value = None  # set to None when data are not available in df_close

#       print(f"No data for next_date_n {next_date_n}, pick's portf value = None")
#       print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

#     else:    
#       p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
#         get_portf_buy(df_close, next_date_n, syms, portf_target=1000, verbose=verbose)
#       s_date, s_price, s_share, s_value = \
#         get_SPY_buy(df_close, next_date_n, portf_target=1000, verbose=verbose)
              
#       print(f"next_date_n pick's portf value = {p_portf_value}")
#       print(f'next_date_n SPY portf value =    {s_value}')

#     date_buy.append(p_date)
#     shares_syms.append(p_ar_sym_share)
#     value_portf.append(p_portf_value)
#     shares_SPY.append(s_share)
#     value_SPY.append(s_value)

#     print('='*20, '\n')

#   return date_buy, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [123]:
# dates = df_mp.date
# symbols = df_mp.symbols

In [124]:
portf_target = 1000
date_buy, shares_syms, value_portf, shares_SPY, value_SPY = calc_portf_value_date_buy(df_mp.date, df_mp.symbols, df_close, portf_target, n=1, verbose=verbose)

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

date = '2023-04-18', l_syms = ['ELF', 'SHV'], ar_price = array([ 92.58000183, 110.29000092]), ar_shares = array([5., 4.]), ar_value = array([462.90000916, 441.16000366]), portf_value = 904.0600128173828
2023-04-18 portf_value = 904.0600128173828
date = '2023-04-18', l_syms = ['SPY'], ar_price = array([414.20999146]), ar_shares = array([2.]), ar_value = array([828.41998291]), portf_value = 828.4199829101562
2023-04-18 portf_value = 828.4199829101562
close_date_n pick's portf value = 904.0600128173828
close_date_n SPY portf value =    828.4199829101562

date = '2023-04-17', l_syms = ['SHV'], ar_price = array([110.26999664]), ar_shares = array([9.]), ar_value = array([992.42996979]), portf_value = 992.4299697875977
2023-04-17 portf_value = 992.4299697875977
date = '2023-04-17', l_syms = ['SPY'], ar_price = array([413.94000244]), ar_shares = array([2.]), ar_value = array([827.

In [125]:
df_mp['date_buy'] = date_buy
df_mp['sh_portf_buy'] = shares_syms
df_mp['$_portf_buy'] = value_portf
df_mp['sh_SPY_buy'] = shares_SPY
df_mp['$_SPY_buy'] = value_SPY
# df_mp.tail()
# df_mp.head()
df_mp

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy
0,2023-04-18,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,NaN,None,NaN
1,2023-04-17,"[30, 60, 120]",sym_freq_8,"['ELF', 'SHV']",2023-04-18,"[5.0, 4.0]",904.060013,[2.0],828.419983
2,2023-04-14,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-17,[9.0],992.429970,[2.0],827.880005
3,2023-04-13,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-14,[9.0],992.340019,[2.0],824.919983
4,2023-04-12,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-13,[9.0],992.519989,[2.0],826.940002
5,2023-04-11,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-12,[9.0],991.980011,[2.0],816.099976
6,2023-04-10,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-11,[9.0],991.889992,[2.0],819.440002
7,2023-04-06,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-10,[9.0],991.710022,[2.0],819.219971
8,2023-04-05,"[30, 60, 120]",sym_freq_9,['SHV'],2023-04-06,[9.0],991.799973,[2.0],818.380005
9,2023-04-04,"[30, 60, 120]",sym_freq_8,"['NVDA', 'SHV']",2023-04-05,"[1.0, 4.0]",709.609985,[2.0],815.200012


In [126]:
def calc_portf_value_date_n(my_dates, my_symbols, df_close, shares_syms, n, verbose=False):
  
  z_date_syms_shares = zip(my_dates, my_symbols, shares_syms)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  # shares_SPY = []  # list of shares of SPY brought on date
  # value_SPY = []  # list of value of SPY shares on date 

  for date, syms, shares in z_date_syms_shares:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_close)

    if close_date_n is None:
      p_date_exec = None
      p_ar_shares = None
      p_value_portf = None  # set to None when data are not available in df_close
      # SPY_shares = None
      # SPY_value = None  # set to None when data are not available in df_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        # print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

    else: 


      if shares is None:
        p_date_exec = None
        p_ar_shares = None
        p_value_portf = None
      else:  
        # p_ar_shares = calc_portf_shares(df_close, next_date_n, syms, portf_target)
        p_date_exec, p_ar_syms, p_ar_price, p_ar_shares, p_ar_value, p_value_portf = \
          calc_portf_value(df_close, close_date_n, syms, shares, verbose)
          # calc_portf_value(df_close, next_date_n, syms, shares, verbose)      


        if verbose:
          print(f"next_date_n pick's portf value = {p_value_portf}")
          # print(f'next_date_n SPY portf value =    {SPY_value}')






    date_exec.append(p_date_exec)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_value_portf)
    # shares_SPY.append(SPY_shares)
    # value_SPY.append(SPY_value)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf   
    

In [127]:
date_exec, shares_syms, value_portf = calc_portf_value_date_n(df_mp.date, df_mp.symbols, df_close, df_mp.sh_portf_buy, n=4, verbose=verbose)

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

date = '2023-04-18', l_syms = ['SHV'], ar_price = array([110.29000092]), ar_shares = array([9.]), ar_value = array([992.61000824]), portf_value = 992.6100082397461
2023-04-18 portf_value = 992.6100082397461
next_date_n pick's portf value = 992.6100082397461

date = '2023-04-17', l_syms = ['SHV'], ar_price = array([110.26999664]), ar_shares = array([9.]), ar_value = array([992.42996979]), portf_value = 992.4299697875977
2023-04-17 portf_value = 992.4299697875977
next_date_n pick's portf value = 992.4299697875977

date = '2023-04-14', l_syms = ['SHV'], ar_price = array([110.26000214]), ar_shares = array([9.]), ar_value = array([992.34001923]), portf_value = 992.3400192260742
2023-04-14 portf_value = 992.3400192260742
next_date_n pick's portf value = 992.340019

In [128]:
# , shares_syms, value_portf
df_mp['date_sell'] = date_exec
df_mp['sh_portf_sell'] = shares_syms
df_mp['$_portf_sell'] = value_portf
df_mp['%_portf_chg'] = (df_mp['$_portf_sell'] / df_mp['$_portf_buy'] - 1) * 100

In [ ]:
# # _c = df_close.loc['2023-04-11']['NVDA']
# _c = df_close.loc['2023-04-11']['SHV']
# _c

In [129]:
df_mp

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg
0,2023-04-18,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,NaN,None,NaN,None,None,NaN,NaN
1,2023-04-17,"[30, 60, 120]",sym_freq_8,"['ELF', 'SHV']",2023-04-18,"[5.0, 4.0]",904.060013,[2.0],828.419983,None,None,NaN,NaN
2,2023-04-14,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-17,[9.0],992.429970,[2.0],827.880005,None,None,NaN,NaN
3,2023-04-13,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-14,[9.0],992.340019,[2.0],824.919983,None,None,NaN,NaN
4,2023-04-12,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-13,[9.0],992.519989,[2.0],826.940002,2023-04-18,[9.0],992.610008,0.009070
5,2023-04-11,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-12,[9.0],991.980011,[2.0],816.099976,2023-04-17,[9.0],992.429970,0.045360
6,2023-04-10,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-11,[9.0],991.889992,[2.0],819.440002,2023-04-14,[9.0],992.340019,0.045371
7,2023-04-06,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-10,[9.0],991.710022,[2.0],819.219971,2023-04-13,[9.0],992.519989,0.081674
8,2023-04-05,"[30, 60, 120]",sym_freq_9,['SHV'],2023-04-06,[9.0],991.799973,[2.0],818.380005,2023-04-12,[9.0],991.980011,0.018153
9,2023-04-04,"[30, 60, 120]",sym_freq_8,"['NVDA', 'SHV']",2023-04-05,"[1.0, 4.0]",709.609985,[2.0],815.200012,2023-04-11,"[1.0, 4.0]",712.529999,0.411496


In [130]:
z_date_syms_shares = zip(df_mp.date, df_mp.symbols, df_mp.sh_portf_buy)
for date, symbols, shares in z_date_syms_shares:
  next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n=1, verbose=False)
  close_date_n = is_date_in_close(next_date_n, df_close)
  print(date, close_date_n, symbols, shares)
  if shares is None:
    print(f'{shares = }')
  else:
    close_date_n, l_syms, ar_price, ar_shares, ar_value, portf_value = \
      calc_portf_value(df_close, close_date_n, symbols, shares, verbose=False)
    print(date, close_date_n, l_syms, ar_price, ar_shares, ar_value, portf_value)



2023-04-18 None ['SHV'] None
shares = None
2023-04-17 2023-04-18 ['ELF', 'SHV'] [5. 4.]
2023-04-17 2023-04-18 ['ELF', 'SHV'] [ 92.58000183 110.29000092] [5. 4.] [462.90000916 441.16000366] 904.0600128173828
2023-04-14 2023-04-17 ['SHV'] [9.]
2023-04-14 2023-04-17 ['SHV'] [110.26999664] [9.] [992.42996979] 992.4299697875977
2023-04-13 2023-04-14 ['SHV'] [9.]
2023-04-13 2023-04-14 ['SHV'] [110.26000214] [9.] [992.34001923] 992.3400192260742
2023-04-12 2023-04-13 ['SHV'] [9.]
2023-04-12 2023-04-13 ['SHV'] [110.27999878] [9.] [992.51998901] 992.5199890136719
2023-04-11 2023-04-12 ['SHV'] [9.]
2023-04-11 2023-04-12 ['SHV'] [110.22000122] [9.] [991.98001099] 991.9800109863281
2023-04-10 2023-04-11 ['SHV'] [9.]
2023-04-10 2023-04-11 ['SHV'] [110.20999908] [9.] [991.88999176] 991.8899917602539
2023-04-06 2023-04-10 ['SHV'] [9.]
2023-04-06 2023-04-10 ['SHV'] [110.19000244] [9.] [991.71002197] 991.7100219726562
2023-04-05 2023-04-06 ['SHV'] [9.]
2023-04-05 2023-04-06 ['SHV'] [110.19999695] [9.] 

In [131]:
df_mp['date_buy'] = date_buy
df_mp['sh_portf_buy'] = shares_syms
df_mp['$_portf_buy'] = value_portf
df_mp['sh_SPY_buy'] = shares_SPY
df_mp['$_SPY_buy'] = value_SPY
# df_mp.tail()
# df_mp.head()
df_mp

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg
0,2023-04-18,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,NaN,None,NaN,None,None,NaN,NaN
1,2023-04-17,"[30, 60, 120]",sym_freq_8,"['ELF', 'SHV']",2023-04-18,None,NaN,[2.0],828.419983,None,None,NaN,NaN
2,2023-04-14,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-17,None,NaN,[2.0],827.880005,None,None,NaN,NaN
3,2023-04-13,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-14,None,NaN,[2.0],824.919983,None,None,NaN,NaN
4,2023-04-12,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-13,[9.0],992.610008,[2.0],826.940002,2023-04-18,[9.0],992.610008,0.009070
5,2023-04-11,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-12,[9.0],992.429970,[2.0],816.099976,2023-04-17,[9.0],992.429970,0.045360
6,2023-04-10,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-11,[9.0],992.340019,[2.0],819.440002,2023-04-14,[9.0],992.340019,0.045371
7,2023-04-06,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-10,[9.0],992.519989,[2.0],819.219971,2023-04-13,[9.0],992.519989,0.081674
8,2023-04-05,"[30, 60, 120]",sym_freq_9,['SHV'],2023-04-06,[9.0],991.980011,[2.0],818.380005,2023-04-12,[9.0],991.980011,0.018153
9,2023-04-04,"[30, 60, 120]",sym_freq_8,"['NVDA', 'SHV']",2023-04-05,"[1.0, 4.0]",712.529999,[2.0],815.200012,2023-04-11,"[1.0, 4.0]",712.529999,0.411496


In [132]:
def get_portf_value_sell(my_dates, my_symbols, df_close, n):
  
  z_date_syms = zip(my_dates, my_symbols)

  date_buy = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date_buy
  value_portf = [] # list of porfolio value on date_buy
  shares_SPY = []  # list of shares of SPY brought on date_buy
  value_SPY = []  # list of value of SPY shares on date_buy 

  for date, syms in z_date_syms:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=verbose)
    close_date_n = is_date_in_close(next_date_n, df_close)

    if close_date_n is None:
      p_date = None
      p_ar_sym_share = None
      p_portf_value = None  # set to None when data are not available in df_close
      s_share = None
      s_value = None  # set to None when data are not available in df_close

      print(f"No data for next_date_n {next_date_n}, pick's portf value = None")
      print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

    else:    
      p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
        get_portf_buy(df_close, next_date_n, syms, portf_target=1000, verbose=verbose)
      s_date, s_price, s_share, s_value = \
        get_SPY_buy(df_close, next_date_n, portf_target=1000, verbose=verbose)
              
      print(f"next_date_n pick's portf value = {p_portf_value}")
      print(f'next_date_n SPY portf value =    {s_value}')

    date_buy.append(p_date)
    shares_syms.append(p_ar_sym_share)
    value_portf.append(p_portf_value)
    shares_SPY.append(s_share)
    value_SPY.append(s_value)

    print('='*20, '\n')

  return date_buy, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [133]:
# date_buy symbols sh_portf_buy	sh_SPY_buy

In [134]:
# # # TODO sell shares MUST equal to buy shares

# date_sell, shares_syms, value_portf, shares_SPY, value_SPY = get_portf_value(df_mp.date, df_mp.symbols, df_close, n=4)
# # df_mp['date_sell'] = date_sell
# # df_mp['sh_portf_sell'] = shares_syms
# # df_mp['$_portf_sell'] = value_portf
# # df_mp['sh_SPY_sell'] = shares_SPY
# # df_mp['$_SPY_sell'] = value_SPY
# # df_mp

In [135]:
print(f'len(df_model_top_picks): {len(df_model_top_picks)}\n')
print(f'date_buy, {len(date_buy) = }, {type(date_buy) = }:\n{date_buy}\n')
print(f'shares_syms, {len(shares_syms) = }, {type(shares_syms) = }:\n{shares_syms}\n')
print(f'value_portf, {len(value_portf) = }, {type(value_portf) = }:\n{value_portf}\n')
print(f'shares_SPY, {len(shares_SPY) = }, {type(shares_SPY) = }:\n{shares_SPY}\n')
print(f'value_SPY, {len(value_SPY) = }, {type(value_SPY) = }:\n{value_SPY}\n')

len(df_model_top_picks): 29

date_buy, len(date_buy) = 29, type(date_buy) = <class 'list'>:
[None, '2023-04-18', '2023-04-17', '2023-04-14', '2023-04-13', '2023-04-12', '2023-04-11', '2023-04-10', '2023-04-06', '2023-04-05', '2023-04-04', '2023-04-03', '2023-03-31', '2023-03-31', '2023-03-30', '2023-03-29', '2023-03-28', '2023-03-28', '2023-03-27', '2023-03-24', '2023-03-23', '2023-03-23', '2023-03-22', '2023-03-22', '2023-03-21', '2023-03-20', '2023-03-20', '2023-03-17', '2023-03-16']

shares_syms, len(shares_syms) = 29, type(shares_syms) = <class 'list'>:
[None, None, None, None, array([9.]), array([9.]), array([9.]), array([9.]), array([9.]), array([1., 4.]), array([5., 1.]), array([3.]), array([9.]), array([3.]), array([9.]), array([9.]), array([9.]), array([10.]), array([1., 4.]), array([9.]), array([9.]), array([16.]), array([9.]), array([11.]), array([8., 4.]), array([8., 4.]), array([11.]), array([8., 4.]), array([8., 4.])]

value_portf, len(value_portf) = 29, type(value_portf)

In [136]:
print(f'len(df_model_top_picks): {len(df_model_top_picks)}\n')
print(f'date_buy, {len(date_buy) = }, {type(date_buy) = }:\n{date_buy}\n')
print(f'shares_syms, {len(shares_syms) = }, {type(shares_syms) = }:\n{shares_syms}\n')
print(f'value_portf, {len(value_portf) = }, {type(value_portf) = }:\n{value_portf}\n')
print(f'shares_SPY, {len(shares_SPY) = }, {type(shares_SPY) = }:\n{shares_SPY}\n')
print(f'value_SPY, {len(value_SPY) = }, {type(value_SPY) = }:\n{value_SPY}\n')

len(df_model_top_picks): 29

date_buy, len(date_buy) = 29, type(date_buy) = <class 'list'>:
[None, '2023-04-18', '2023-04-17', '2023-04-14', '2023-04-13', '2023-04-12', '2023-04-11', '2023-04-10', '2023-04-06', '2023-04-05', '2023-04-04', '2023-04-03', '2023-03-31', '2023-03-31', '2023-03-30', '2023-03-29', '2023-03-28', '2023-03-28', '2023-03-27', '2023-03-24', '2023-03-23', '2023-03-23', '2023-03-22', '2023-03-22', '2023-03-21', '2023-03-20', '2023-03-20', '2023-03-17', '2023-03-16']

shares_syms, len(shares_syms) = 29, type(shares_syms) = <class 'list'>:
[None, None, None, None, array([9.]), array([9.]), array([9.]), array([9.]), array([9.]), array([1., 4.]), array([5., 1.]), array([3.]), array([9.]), array([3.]), array([9.]), array([9.]), array([9.]), array([10.]), array([1., 4.]), array([9.]), array([9.]), array([16.]), array([9.]), array([11.]), array([8., 4.]), array([8., 4.]), array([11.]), array([8., 4.]), array([8., 4.])]

value_portf, len(value_portf) = 29, type(value_portf)

In [137]:
print(f'len(df_model_top_picks: {len(df_model_top_picks)}\n')
print(f'date_buy, {len(date_buy) = }, {type(date_buy) = }:\n{date_buy}\n')
print(f'shares_syms, {len(shares_syms) = }, {type(shares_syms) = }:\n{shares_syms}\n')
print(f'value_portf, {len(value_portf) = }, {type(value_portf) = }:\n{value_portf}\n')
print(f'shares_SPY, {len(shares_SPY) = }, {type(shares_SPY) = }:\n{shares_SPY}\n')
print(f'value_SPY, {len(value_SPY) = }, {type(value_SPY) = }:\n{value_SPY}\n')

len(df_model_top_picks: 29

date_buy, len(date_buy) = 29, type(date_buy) = <class 'list'>:
[None, '2023-04-18', '2023-04-17', '2023-04-14', '2023-04-13', '2023-04-12', '2023-04-11', '2023-04-10', '2023-04-06', '2023-04-05', '2023-04-04', '2023-04-03', '2023-03-31', '2023-03-31', '2023-03-30', '2023-03-29', '2023-03-28', '2023-03-28', '2023-03-27', '2023-03-24', '2023-03-23', '2023-03-23', '2023-03-22', '2023-03-22', '2023-03-21', '2023-03-20', '2023-03-20', '2023-03-17', '2023-03-16']

shares_syms, len(shares_syms) = 29, type(shares_syms) = <class 'list'>:
[None, None, None, None, array([9.]), array([9.]), array([9.]), array([9.]), array([9.]), array([1., 4.]), array([5., 1.]), array([3.]), array([9.]), array([3.]), array([9.]), array([9.]), array([9.]), array([10.]), array([1., 4.]), array([9.]), array([9.]), array([16.]), array([9.]), array([11.]), array([8., 4.]), array([8., 4.]), array([11.]), array([8., 4.]), array([8., 4.])]

value_portf, len(value_portf) = 29, type(value_portf) 

In [138]:
# df_model_picks_perf = df_model_top_picks.copy()
_df = df_model_top_picks.copy()
_df.head()

,date,days_lookback,syms_freq,symbols
0,2023-04-18,"[30, 60, 120]",sym_freq_8,['SHV']
1,2023-04-17,"[30, 60, 120]",sym_freq_8,"['ELF', 'SHV']"
2,2023-04-14,"[30, 60, 120]",sym_freq_8,['SHV']
3,2023-04-13,"[30, 60, 120]",sym_freq_8,['SHV']
4,2023-04-12,"[30, 60, 120]",sym_freq_8,['SHV']


In [139]:
_df['date_buy'] = date_buy
_df['sh_syms'] = shares_syms
_df['$_portf_buy'] = value_portf
_df['sh_SPY'] = shares_SPY
_df['$_SPY_buy'] = value_SPY
_df.head()
# _df.tail()

,date,days_lookback,syms_freq,symbols,date_buy,sh_syms,$_portf_buy,sh_SPY,$_SPY_buy
0,2023-04-18,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,NaN,None,NaN
1,2023-04-17,"[30, 60, 120]",sym_freq_8,"['ELF', 'SHV']",2023-04-18,None,NaN,[2.0],828.419983
2,2023-04-14,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-17,None,NaN,[2.0],827.880005
3,2023-04-13,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-14,None,NaN,[2.0],824.919983
4,2023-04-12,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-13,[9.0],992.610008,[2.0],826.940002


In [140]:
my_dates = ['2013-01-01', '2013-01-02', '2013-01-12', '2023-12-29', '2023-12-30', '2023-04-13']
n = 1
dates_NYSE = dates_NYSE_2013_2023

idx_close = df_close.index.strftime('%Y-%m-%d')

for date in my_dates:
    date_n = get_NYSE_date_n(date, dates_NYSE, n, verbose=False)
    print(f'date: {date},  n: {n},  date_n: {date_n}')
    if date_n in idx_close:
        print(f'date_n: {date_n} is in df_close\n')
    else:
        print(f'date_n: {date_n} is not in df_close\n')


date: 2013-01-01,  n: 1,  date_n: None
date_n: None is not in df_close

date: 2013-01-02,  n: 1,  date_n: 2013-01-03
date_n: 2013-01-03 is not in df_close

date: 2013-01-12,  n: 1,  date_n: None
date_n: None is not in df_close

date: 2023-12-29,  n: 1,  date_n: None
date_n: None is not in df_close

date: 2023-12-30,  n: 1,  date_n: None
date_n: None is not in df_close

date: 2023-04-13,  n: 1,  date_n: 2023-04-14
date_n: 2023-04-14 is in df_close

